In [93]:
import pandas as pd
import numpy as np
from pathlib import Path
import xml.etree.ElementTree as ET

In [101]:
DATA_PATH = Path('data')
MDB_XML_PATH = DATA_PATH / 'MDB_STAMMDATEN.XML'
DF_MDB_PATH = DATA_PATH / 'DF_MDB.csv'

# parse xml data into dataFrame / .csv fomat

In [99]:
def get_mdb_data(mdb_xml_path = MDB_XML_PATH):
    list_of_mdb = []
    MDB_STAMMDATEN = ET.parse(mdb_xml_path)
    root = MDB_STAMMDATEN.getroot()
    
    for mdb in root.findall('MDB'):
        mdb_dict = {}
        ID, NAMEN, BIOGRAFISCHE_ANGABEN, WAHLPERIODEN = mdb[0], mdb[1], mdb[2], mdb[3]

        mdb_dict['ID'] = ID.text
        mdb_dict['NACHNAME'] = NAMEN.find('NAME').find('NACHNAME').text  
        mdb_dict['VORNAME'] = NAMEN.find('NAME').find('VORNAME').text  

        #print(mdb.tag, mdb.text, mdb.findall('NACHNAME'))
        for element in BIOGRAFISCHE_ANGABEN:
            mdb_dict[element.tag] = element.text

        mdb_dict['ANZ_WAHLPERIODEN'] = len(list(WAHLPERIODEN))

        for wahlperiode in WAHLPERIODEN:
            mdb_dict[int(wahlperiode.find('WP').text)] = 1

        list_of_mdb.append(mdb_dict)
    return pd.DataFrame(list_of_mdb)

In [100]:
df_mdb = get_mdb_data(MDB_XML_PATH)
print(df_mdb.shape)
df_mdb.head()

(4089, 34)


,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,RELIGION,...,2,12,13,14,15,16,1,17,18,19
0,11000001,Abelein,Manfred,20.10.1930,Stuttgart,None,17.01.2008,männlich,keine Angaben,katholisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11000002,Achenbach,Ernst,09.04.1909,Siegen,None,02.12.1991,männlich,"verheiratet, 3 Kinder",evangelisch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11000003,Ackermann,Annemarie,26.05.1913,Parabutsch,Jugoslawien,18.02.1994,weiblich,"verheiratet, 5 Kinder",katholisch,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11000004,Ackermann,Else,06.11.1933,Berlin,None,14.09.2019,weiblich,ledig,evangelisch,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11000005,Adam,Ulrich,09.06.1950,"Teterow, Kr. Teterow, Bezirk Neubrandenburg",None,None,männlich,"verheiratet, 2 Kinder",evangelisch,...,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN


In [103]:
df_mdb.to_csv(DF_MDB_PATH)

# read in data. exloratory data analysis

In [128]:
df_mdb = pd.read_csv(DF_MDB_PATH)

In [130]:
df_mdb.tail()

,Unnamed: 0,ID,NACHNAME,VORNAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,STERBEDATUM,GESCHLECHT,FAMILIENSTAND,...,2,12,13,14,15,16,1,17,18,19
4084,4084,11004958,Nölke,Matthias,08.01.1980,Kassel,NaN,NaN,männlich,ledig,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4085,4085,11004959,Martin,Dorothee,21.01.1978,NaN,NaN,NaN,weiblich,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4086,4086,11004960,Wetzel,Wolfgang,11.05.1968,Schlema,NaN,NaN,männlich,"verheiratet, 2 Kinder",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4087,4087,11004961,Natterer,Christian,19.02.1981,Tettnang / Deutschland,NaN,NaN,männlich,"verheiratet, 1 Kind",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4088,4088,11004962,Dahmen,Janosch,06.09.1981,NaN,NaN,NaN,männlich,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [131]:
# wie - nur 2 Geschlechter? for the records: heute ist der 02.05.2021
df_mdb[['ID', 'GESCHLECHT']].groupby('GESCHLECHT').count().sort_values(by='ID', ascending=False).head()

,ID
GESCHLECHT,
männlich,3231
weiblich,858


In [132]:
# auch hier: such diversity. wow
df_mdb[['ID', 'RELIGION']].groupby('RELIGION').count().sort_values(by='ID', ascending=False).head(n=16)

,ID
RELIGION,
ohne Angaben,1214
evangelisch,1154
katholisch,870
römisch-katholisch,316
evangelisch-lutherisch,138
konfessionslos,118
evangelisch-reformiert,15
religionslos,7
Atheistin,6


In [127]:
# so viele Busfahrer und Krankenschwestern!
df_mdb[['ID', 'BERUF']].groupby('BERUF').count().sort_values(by='ID', ascending=False).head(16).head(16)

,ID
BERUF,
Rechtsanwalt,182
Geschäftsführer,80
Landwirt,79
Angestellter,63
Rechtsanwalt und Notar,47
Kaufmann,45
Journalist,43
Hausfrau,41
Dipl.-Ingenieur,37


In [133]:
# gehen sie weiter. hier gibt es kein Klischee zu erkennen!
df_mdb[['ID', 'FAMILIENSTAND']].groupby('FAMILIENSTAND').count().sort_values(by='ID', ascending=False).head(16)

,ID
FAMILIENSTAND,
"verheiratet, 2 Kinder",932
verheiratet,565
"verheiratet, 3 Kinder",550
keine Angaben,540
"verheiratet, 1 Kind",465
"verheiratet, 4 Kinder",251
ledig,202
"verheiratet, 5 Kinder",98
"verheiratet, 6 Kinder",47
